<a href="https://colab.research.google.com/github/FNS-Division/GIGA-applied-geospatial-tools/blob/main/get_giga_maps_data_from_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data from Giga Maps

In [ ]:
import requests
import pycountry
import ipywidgets as widgets
import pandas as pd
import geopandas as gpd
from IPython.display import display, clear_output

In [ ]:
# @title Select country
# Create dropdown: country name shown, ISO3 code stored
country_dropdown = widgets.Dropdown(
    options=sorted([(c.name, c.alpha_3) for c in pycountry.countries]),
    description='Country:',
    value='BRA'
)

display(country_dropdown)

In [ ]:
#Choose country to get school location data using ISO code
country_code = ""

In [ ]:
# Set your unique API key (from Giga maps)
api_key = ""

In [ ]:
# URL for API endpoint (using selected country)
BASE_URL = f"https://uni-ooi-giga-maps-service.azurewebsites.net/api/v1/schools_location/country/{country_code}"

#Headers
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}

In [ ]:
# Parameters (see Giga Maps API docs)
# Requests need to be paginated- split up into multiple requests to prevent overloading the API
page = 1

# Current max page size is 100, meaning each request sends 
size = 1000

In [ ]:
# Creates a list to store all the responses from API requests
all_schools = []

# Loop through pages until all school location data has been collected
while True:
    params = {
        "page": page,
        "size": size
    }
    
   # Request schools from the API endpoint 
    response = requests.get(BASE_URL, headers=headers, params=params)

    # Ensure that the response from the API is correct. With an incorrect response, no more requests will be sent
    if response.status_code != 200:
        print(f"Request failed at page {page}: {response.status_code}")
        break

    # Extract the schools returned from the API
    data = response.json()['data']

    # Check that the most recent request receives data in the response
    if not data or len(data) == 0:
        print(f"No more data returned at page {page}.")
        break

    #Continuously add newly fetched schools to the all_schools list
    all_schools.extend(data)
    clear_output(wait=True)
    display(f"Fetched page {page}, items received: {len(data)}")
    page += 1

In [ ]:
school_data = pd.DataFrame(all_schools)

In [ ]:
school_data.columns

In [ ]:
school_data = gpd.GeoDataFrame(school_data, geometry=gpd.points_from_xy(school_data.longitude, school_data.latitude), crs="4326")

In [ ]:
# Save the filtered data to a CSV file
school_data.to_file("school_data.geojson", driver="GeoJSON")